## 1. Иморты и конфигурация

In [ ]:
import torch
import pandas as pd
import re
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTConfig, SFTTrainer

MODEL_PATH = "./model2"
DATASET_PATH = "anecdotes_dataset.csv"
PREFIXES_PATH = "prefixes.txt"
OUTPUT_FILE = "generated_anecdotes.txt"
ADAPTER_OUTPUT_DIR = "smollm_anecdotes_lora"

dtype = torch.float16 if torch.cuda.get_device_capability()[0] < 8 else torch.bfloat16
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device} with dtype: {dtype}")

## 2. Подготовка данных

In [ ]:
print("Загрузка и подготовка данных...")

df = pd.read_csv(DATASET_PATH)
df = df[:2000]

dataset = Dataset.from_pandas(df[['joke']])

## 3. Загрузка модели и токенайзера

In [ ]:
print("Загрузка модели...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    local_files_only=True,
    torch_dtype=dtype,
    device_map="auto"
)

## 4. Настройка LoRA

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"] # Таргетим все линейные слои для лучшего качества
)

## 5. Обучение (SFT)

In [ ]:
sft_config = SFTConfig(
    output_dir=ADAPTER_OUTPUT_DIR,
    dataset_text_field="joke",
    max_length=192,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=(dtype == torch.float16),
    bf16=(dtype == torch.bfloat16),
    optim="adamw_torch",
    save_strategy="no",
    report_to="none",
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=sft_config,
    processing_class=tokenizer,
    peft_config=peft_config,
)

print("Начало обучения...")
trainer.train()

trainer.model.save_pretrained(ADAPTER_OUTPUT_DIR)
print("Обучение завершено")

## 6. Генерация анекдотов ☺

In [ ]:
print("Генерация анекдотов по затравкам...")

model.eval()

with open(PREFIXES_PATH, "r", encoding="utf-8") as f:
    prefixes_raw = f.readlines()

generated_jokes = []

for line in prefixes_raw:
    line = line.strip()
    if not line: continue

    parts = line.split(' ', 1)
    if len(parts) > 1 and parts[0].isdigit():
        prompt_text = parts[1]
    else:
        prompt_text = line

    inputs = tokenizer(prompt_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    clean_joke = full_text.replace("\n", " ").strip()
    generated_jokes.append(clean_joke)
    print(f"Сгенерировано: {clean_joke[:50]}...")

## 7. Сохранение результата

In [ ]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for joke in generated_jokes:
        f.write(joke + "\n")

print(f"Анекдоты сохранены в файл {OUTPUT_FILE}")